# PROMETHEE
---
_**P**reference **R**anking **O**rganization **Meth**od for **E**nrichment **E**valuation_

In [1]:
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

## 1. Introduction
---
The **PROMETHEE** (_Preference Ranking Organization Method for Enrichment Evaluation_) method is technique for prioritizing elements based on circumstances. There are a bunch of variations for this algorithm, but here we will focus on **PROMETHEE II**, which basically depends on the _priority weights_ defined by the decisor.

### 1.1. Applications
---
The _PROMETHEE II method_ can be used in dicision situations where priorization is the main objective, such as:

- Urban planning
- Human resource management
- Choice of products
- Feature priorization

For example, let's consider a set of elements with **3 criteria** and **5 alternatives**, generated randomly:

In [3]:
np.random.seed(0)

m = 5
C = np.random.randint(100, 500, m)
df = pd.DataFrame({
    "Criteria 1": C,
    "Criteria 2": (C**(np.random.random(m)/2)).astype(int),
    "Criteria 3": (C*(1 + np.random.random(m))//2).astype(int)
})

df.index = [f'Alternative {i}' for i in np.arange(m)+1]

df.style.background_gradient()

,Criteria 1,Criteria 2,Criteria 3
Alternative 1,272,11,143
Alternative 2,147,8,93
Alternative 3,217,5,160
Alternative 4,292,2,264
Alternative 5,423,2,313


## 2. Method steps
---

### 2.1. Objectives and weights
---
The first step is the definition of **objectives** and **weights**. The _objective_ irefers to **minimizing** or **maximizing** the criteria. The _weight_ defines the degree of importance of each criterion. All of them must be defined by the **decision maker**, before proceeding with the method.

In [4]:
# Criteria 1: {objective: max, weight: 3},
# Criteria 2: {objective: min, weight: 2},
# Criteria 3: {objective: max, weight: 3}

OBJECTIVES = pd.Series(["max", "min", "max"])
WEIGHTS = pd.Series([3, 2, 3])
WEIGHTS = WEIGHTS/WEIGHTS.sum()

OBJECTIVES.index = WEIGHTS.index = [
    "Criteria 1", "Criteria 2", "Criteria 3"
]

OBJECTIVES, WEIGHTS

(Criteria 1    max
 Criteria 2    min
 Criteria 3    max
 dtype: object,
 Criteria 1    0.375
 Criteria 2    0.250
 Criteria 3    0.375
 dtype: float64)

### 2.2. Criteria normalization
---
The second step is data normalization, which consists of transforming all criteria and placing them on equivalence. For criteria the objective is the **maximization**, the formula is:

$$ \large
_{max} a_{ij}' = \frac{a_{ij} - \min(a_{ij})}{\max(a_{ij}) - \min(a_{ij})}
$$

For criteria the objective is the **minimization**, the formula is:

$$ \large
\begin{aligned}
_{min} a_{ij}' &= \frac{\max(a_{ij}) - a_{ij}}{\max(a_{ij}) - \min(a_{ij})} \\
&= 1 - \frac{a_{ij} - \min(a_{ij})}{\max(a_{ij}) - \min(a_{ij})}
\end{aligned}
$$

In [5]:
df_n = (df - df.min())/(df.max() - df.min())

df_n

,Criteria 1,Criteria 2,Criteria 3
Alternative 1,0.452899,1.000000,0.227273
Alternative 2,0.000000,0.666667,0.000000
Alternative 3,0.253623,0.333333,0.304545
Alternative 4,0.525362,0.000000,0.777273
Alternative 5,1.000000,0.000000,1.000000


In [6]:
for k, v in OBJECTIVES.items():
    if v == "max": continue
    df_n[k] = 1 - df_n[k]

df_n

,Criteria 1,Criteria 2,Criteria 3
Alternative 1,0.452899,0.000000,0.227273
Alternative 2,0.000000,0.333333,0.000000
Alternative 3,0.253623,0.666667,0.304545
Alternative 4,0.525362,1.000000,0.777273
Alternative 5,1.000000,1.000000,1.000000


### 2.3. Pairwise differentiation
---
The third step is a pairwise comparison between alternatives:

$$ \large
d_{k}(a_{i},a_{j})=f_{k}(a_{i})-f_{k}(a_{j})
$$

where $ \large d_{k}(a_{i},a_{j})$ is the difference between the alternatives $ \large a_i$ and $ \large a_j$, with their respective values $ \large f_{k}(a_i)$ and $ \large f_{k}(a_j)$ for the criterion $ \large k$.

In [7]:
n = len(df_n)

indices = [(i, j) for i in range(n) for j in range(n)]
df_diff = [df_n.iloc[i] - df_n.iloc[j] for i, j in indices]
df_diff = pd.DataFrame(data=df_diff)
df_diff.index = [f'{i} - {j}' for i in df.index for j in df.index]

df_diff

,Criteria 1,Criteria 2,Criteria 3
Alternative 1 - Alternative 1,0.000000,0.000000,0.000000
Alternative 1 - Alternative 2,0.452899,-0.333333,0.227273
Alternative 1 - Alternative 3,0.199275,-0.666667,-0.077273
Alternative 1 - Alternative 4,-0.072464,-1.000000,-0.550000
Alternative 1 - Alternative 5,-0.547101,-1.000000,-0.772727
Alternative 2 - Alternative 1,-0.452899,0.333333,-0.227273
Alternative 2 - Alternative 2,0.000000,0.000000,0.000000
Alternative 2 - Alternative 3,-0.253623,-0.333333,-0.304545
Alternative 2 - Alternative 4,-0.525362,-0.666667,-0.777273
Alternative 2 - Alternative 5,-1.000000,-0.666667,-1.000000


### 2.4. Preference degree
---
The fourth step is to calculate _multicriteria preference degree_. Before that, we need to specify the **preference function**, which basically translate the difference into a unicriterion preference degree. There are a bunch of [preference functions](https://en.wikipedia.org/wiki/Preference_ranking_organization_method_for_enrichment_evaluation#Promethee_preference_functions) we will use the _linear preference function_, defined as:

$$ \large
P(x)= 
\begin{cases}
0 &, \text{if } x \leq 0 \\
x &, \text{otherwise}
\end{cases}
$$

Now we can associate each criteria with its _preference function_ and _weights_ to calculate the _multicriteria preference degree_, in order to globally compare every couple of actions:

$$ \large
\pi(a_i,a_j) = \sum_k P_k[d_k(a_i,a_j)] \cdot w_k
$$

In [8]:
df_pref = df_diff.applymap(lambda x: 0 if x <= 0 else x)
df_pref = np.sum(df_pref*WEIGHTS, axis=1)
df_pref = df_pref.to_frame("pref_m")

df_pref[["index_", "column_"]] = [[ee for ee in e.split(" - ")] for e in df_pref.index]
df_pref = df_pref.pivot_table(values="pref_m", index="index_", columns="column_")
df_pref.index.name = None
df_pref.columns.name = None

df_pref

,Alternative 1,Alternative 2,Alternative 3,Alternative 4,Alternative 5
Alternative 1,0.000000,0.255064,0.074728,0.000000,0.0
Alternative 2,0.083333,0.000000,0.000000,0.000000,0.0
Alternative 3,0.195644,0.292647,0.000000,0.000000,0.0
Alternative 4,0.483424,0.655155,0.362508,0.000000,0.0
Alternative 5,0.744936,0.916667,0.624020,0.261512,0.0


### 2.5. Preference flow
---
The fifth step is the calculation of _preference flow_, which is basically the final **score** of PROMETHEE method that position every action with respect to all the other actions. Before that, we have to find the **positive preference flow** and **negative preference flow**, as follows:

$$ \large
\phi^+(a) = \frac{1}{n-1} \sum_{b \in A} \pi(a,b)
$$

$$ \large
\phi^-(a) = \frac{1}{n-1} \sum_{b \in A} \pi(b,a)
$$

Having that, we are able to aggregate them into the **net preference flow**:

$$ \large
\phi(a) = \phi^+(a) - \phi^-(a)
$$

In [9]:
n = len(df)

neg_flow = df_pref.sum(axis=0)/(n - 1)
pos_flow = df_pref.sum(axis=1)/(n - 1)
net_flow = pos_flow - neg_flow

df_net_flow = net_flow.to_frame("net_flow")
df_flow = df.join(df_net_flow)

df_flow

,Criteria 1,Criteria 2,Criteria 3,net_flow
Alternative 1,272,11,143,-0.294386
Alternative 2,147,8,93,-0.509050
Alternative 3,217,5,160,-0.143242
Alternative 4,292,2,264,0.309894
Alternative 5,423,2,313,0.636784


### 2.6. Ranking
---
Finally, the last step is to define a **ranking** by ordering the _preference flow_ in descending way.

In [10]:
df_flow["ranking"] = (
    df_flow["net_flow"]
    .rank(
        ascending=False,
        method="first"
    )
)

df_rank = df_flow.sort_values("ranking")

(
    df_rank.sort_values("ranking")
    .style.background_gradient(
        subset=list(df.columns)
    )
)

,Criteria 1,Criteria 2,Criteria 3,net_flow,ranking
Alternative 5,423,2,313,0.636784,1.000000
Alternative 4,292,2,264,0.309894,2.000000
Alternative 3,217,5,160,-0.143242,3.000000
Alternative 1,272,11,143,-0.294386,4.000000
Alternative 2,147,8,93,-0.509050,5.000000


## 3. Examples
---

### 3.1. Investment portfolio
---
As an example, let's build an investment portfolio by priorizing companies based on their performance. The data was taken from [slickcharts](https://www.slickcharts.com/sp500/performance) we will consider 5 criteria: **YTD Return** (percent), **Market Cap** (in Billions), **Volume**, **DY TTM** and **Debt** (in Billions). The _debt_ data was taken from [companiesmarketcap](https://companiesmarketcap.com/companies-with-the-highest-debt/).

In [11]:
df_ex1 = pd.DataFrame({
    "YTD Return": [13.42, 10.92, 120.54, 31.7, 22.24, 23.76],
    "Market Cap": [3348.35, 3100.4, 2686.57, 684.36, 2102.06, 362.15],
    "Volume": [67_773_676, 16_929_695, 370_702_780, 20_028_287, 17_912_584, 1_232_911],
    "DY TTM": [0.44, 0.7, 0.12, 13.84, 0.12, 0.66],
    "Debt": [104.59, 79.91, 11.23, 36.96, 29.42, 9.3]
})

df_ex1.index = [
    "Apple Inc.", 
    "Microsoft Corp.",
    "Nvidia Corp.",
    "Broadcom Inc.",
    "Alphabet Inc.",
    "Costco Wholesale Corp."
]

df_ex1.style.background_gradient()

,YTD Return,Market Cap,Volume,DY TTM,Debt
Apple Inc.,13.420000,3348.350000,67773676,0.440000,104.590000
Microsoft Corp.,10.920000,3100.400000,16929695,0.700000,79.910000
Nvidia Corp.,120.540000,2686.570000,370702780,0.120000,11.230000
Broadcom Inc.,31.700000,684.360000,20028287,13.840000,36.960000
Alphabet Inc.,22.240000,2102.060000,17912584,0.120000,29.420000
Costco Wholesale Corp.,23.760000,362.150000,1232911,0.660000,9.300000


In [12]:
# Objective and weights
# YTD Return: {objective: max, weight: 3},
# Market Cap: {objective: max, weight: 4},
#     Volume: {objective: max, weight: 2},
#     DY TTM: {objective: max, weight: 6},
#       Debt: {objective: min, weight: 5}

OBJECTIVES_ex1 = pd.Series(["max", "max", "max", "max", "min"])
WEIGHTS_ex1 = pd.Series([3, 4, 2, 6, 5])
WEIGHTS_ex1 = WEIGHTS_ex1/WEIGHTS_ex1.sum()

OBJECTIVES_ex1.index = WEIGHTS_ex1.index = [
    "YTD Return", "Market Cap", "Volume", "DY TTM", "Debt"
]

# Criteria normalization
df_ex1_n = (df_ex1 - df_ex1.min())/(df_ex1.max() - df_ex1.min())

for k, v in OBJECTIVES_ex1.items():
    if v == "max": continue
    df_ex1_n[k] = 1 - df_ex1_n[k]

# Pairwise differentiation
n = len(df_ex1_n)

indices = [(i, j) for i in range(n) for j in range(n)]
df_ex1_diff = [df_ex1_n.iloc[i] - df_ex1_n.iloc[j] for i, j in indices]
df_ex1_diff = pd.DataFrame(data=df_ex1_diff)
df_ex1_diff.index = [f'{i} - {j}' for i in df_ex1.index for j in df_ex1.index]

# Preference degree
df_ex1_pref = df_ex1_diff.applymap(lambda x: 0 if x <= 0 else x)
df_ex1_pref = np.sum(df_ex1_pref*WEIGHTS_ex1, axis=1)
df_ex1_pref = df_ex1_pref.to_frame("pref_m")

df_ex1_pref[["index_", "column_"]] = [[ee for ee in e.split(" - ")] for e in df_ex1_pref.index]
df_ex1_pref = df_ex1_pref.pivot_table(values="pref_m", index="index_", columns="column_")
df_ex1_pref.index.name = None
df_ex1_pref.columns.name = None

# Preference flow
n = len(df_ex1)

neg_flow = df_ex1_pref.sum(axis=0)/(n - 1)
pos_flow = df_ex1_pref.sum(axis=1)/(n - 1)
net_flow = pos_flow - neg_flow

df_ex1_net_flow = net_flow.to_frame("net_flow")
df_ex1_flow = df_ex1.join(df_ex1_net_flow)

# Ranking
df_ex1_flow["ranking"] = (
    df_ex1_flow["net_flow"]
    .rank(
        ascending=False,
        method="first"
    )
)

df_ex1_rank = df_ex1_flow.sort_values("ranking")

(
    df_ex1_rank.sort_values("ranking")
    .style.background_gradient(
        subset=list(df_ex1.columns)
    )
)

,YTD Return,Market Cap,Volume,DY TTM,Debt,net_flow,ranking
Nvidia Corp.,120.540000,2686.570000,370702780,0.120000,11.230000,0.322782,1.000000
Broadcom Inc.,31.700000,684.360000,20028287,13.840000,36.960000,0.181085,2.000000
Alphabet Inc.,22.240000,2102.060000,17912584,0.120000,29.420000,-0.057457,3.000000
Costco Wholesale Corp.,23.760000,362.150000,1232911,0.660000,9.300000,-0.122702,4.000000
Microsoft Corp.,10.920000,3100.400000,16929695,0.700000,79.910000,-0.139866,5.000000
Apple Inc.,13.420000,3348.350000,67773676,0.440000,104.590000,-0.183842,6.000000
